# 3CX Supply Chain
> Walkthrough
+ categories: [traffic-analysis, malicious traffic]
+ source: [CybersDefenderos](https://cyberdefenders.org/blueteam-ctf-challenges/3cx-supply-chain/)

## Overview
A large multinational corporation heavily relies on the 3CX software for phone communication, making it a critical component of their business operations. After a recent update to the 3CX Desktop App, antivirus alerts flag sporadic instances of the software being wiped from some workstations while others remain unaffected. Dismissing this as a false positive, the IT team overlooks the alerts, only to notice degraded performance and strange network traffic to unknown servers. Employees report issues with the 3CX app, and the IT security team identifies unusual communication patterns linked to recent software updates.

As the threat intelligence analyst, it's your responsibility to examine this possible supply chain attack. Your objectives are to uncover how the attackers compromised the 3CX app, identify the potential threat actor involved, and assess the overall extent of the incident. 

Tools:  
+ VirusTotal

---

**Q1:** Understanding the scope of the attack and identifying which versions exhibit malicious behavior is crucial for making informed decisions if these compromised versions are present in the organization. How many versions of 3CX running on Windows have been flagged as malware?  
**A:**  2  
+ [3CX](https://www.3cx.com/blog/news/desktopapp-security-alert/)
+ [Reliaquest](https://www.reliaquest.com/blog/3cx-trojan-attack/)
+ [ITPro](https://www.itpro.com/security/malware/370353/3cx-ceo-state-sponsored-hackers-behind-supply-chain-malware-attack)


**Q2:** Determining the age of the malware can help assess the extent of the compromise and track the evolution of malware families and variants. What's the UTC creation time of the .msi malware?   
**A:**  '2023-03-13 06:33:26 UTC'   
![](../../images/3cx-msi-vt-ct.png)

**Q3:** Executable files (.exe) are frequently used as primary or secondary malware payloads, while dynamic link libraries (.dll) often load malicious code or enhance malware functionality. Analyzing files deposited by the Microsoft Software Installer (.msi) is crucial for identifying malicious files and investigating their full potential. Which malicious DLLs were dropped by the .msi file?  
**A:** 'ffmpeg.dll d3dcompiler_47.dll'

**Tools required**: 
+ msitools
+ (Optional) [didier-stevens-suite](https://blog.didierstevens.com/didier-stevens-suite/)


1. Extract msi contents:  
`msiextract 3CXDesktopApp-18.12.416.msi`

2. Locate DLLs:

   `find 3CXDesktopApp-18.12.416 -name "*.dll"`

   ./d3dcompiler_47.dll  
   ./ffmpeg.dll  
   ./libEGL.dll  
   ./libGLESv2.dll  
   ./vk_swiftshader.dll  
   ./vulkan-1.dll


3. DLL Identification:  
_After uploading each DLL to VirusTotal; the following DLLs seem to be malicious._

./d3dcompiler_47.dll
+ SHA1:  20d554a80d759c50d6537dd7097fed84dd258b3e 

./ffmpeg.dll    
+ SHA1:  bf939c9c261d27ee7bb92325cc588624fca75429

**Q4:**  Recognizing the persistence techniques used in this incident is essential for current mitigation strategies and future defense improvements. What is the MITRE sub-technique ID employed by the .msi files to load the malicious DLL?  

> The MITRE ATT&CK framework uses unique identifiers for each technique and sub-technique to systematically categorize adversary behaviors. These identifiers are known as "Technique IDs" and "Sub-technique IDs."

**A:**  'T1574.002'  
![](../../images/mitre_T1574-002.png)

**Q5:** Recognizing the malware type (threat category) is essential to your investigation, as it can offer valuable insight into the possible malicious actions you'll be examining. What is the malware family of the two malicious DLLs?  
**A:**  
./d3dcompiler_47.dll  (**Trojan**)  
./ffmpeg.dll  (**Trojan**)  

**Q6:** As a threat intelligence analyst conducting dynamic analysis, it's vital to understand how malware can evade detection in virtualized environments or analysis systems. This knowledge will help you effectively mitigate or address these evasive tactics. What is the MITRE ID for the virtualization/sandbox evasion techniques used by the two malicious DLLs?  

**A:**  'T1497'  
![](../../images/mitre_T1497.png)

**Q7:**  When conducting malware analysis and reverse engineering, understanding anti-analysis techniques is vital to avoid wasting time. Which hypervisor is targeted by the anti-analysis techniques in the ffmpeg.dll file?  
**A:**  'VMWare'  
![](../../images/anti-analysis-vm-type.png)

**Q8:**  Identifying the cryptographic method used in malware is crucial for understanding the techniques employed to bypass defense mechanisms and execute its functions fully. What encryption algorithm is used by the ffmpeg.dll file?  
**A:**  'RC4'  
![](../../images/encryption-type.png)
> RC4 is a fast, symmetric key stream cipher used for encrypting data. It uses the same key for both encryption and decryption, making it efficient for real-time applications. Despite its speed, RC4 has known vulnerabilities and is considered less secure than modern encryption methods, so it's not recommended for sensitive data.

**Q9:** As an analyst, you've recognized some TTPs involved in the incident, but identifying the APT group responsible will help you search for their usual TTPs and uncover other potential malicious activities. Which group is responsible for this attack?  

> TTP: Tactics, techniques, and procedures.

**A:** Lazarus